# Appendix 1 : Filter the whole OpenFoodFacts database

In [22]:
import itertools
import pandas as pd
import numpy as np

In [4]:
OFF_FILE = "/home/mathieu/datasets/openfoodfacts/2021-08-16/en.openfoodfacts.org.products.csv"

The OpenFoodFacts website links to a file that [describes the data fields](https://static.openfoodfacts.org/data/data-fields.txt).

The first important information is that the OpenFoodFacts CSV file is encoded in UTF-8 and uses the TAB (`\t`) character as delimiter.

We will read the first 1,000 rows and see what data types are inferred by `read_csv`.

Then we will assign explicit data types or convert values following a few guidelines :

* Most of the columns in OpenFoodFacts contain nutritional values. Their name ends with `_100g` and their values are [decimal numerals](https://en.wikipedia.org/wiki/Decimal) ;
* Columns ending with `_t` contain [Unix times](https://en.wikipedia.org/wiki/Unix_time) ;
* Columns ending with `_datetime` contain date and time in [Coordinated Universal Time](https://en.wikipedia.org/wiki/Coordinated_Universal_Time) in the [ISO 8601 format](https://en.wikipedia.org/wiki/ISO_8601) ;
* Columns ending with `_tags` are comma separated lists of (string) values.

The `object` data type is the default data type in pandas, that is used whenever a column has non-numerical values.
There is however a `string` data type that you can explicitly set for columns that contain textual values (strings).

In [84]:
def 

def load_off_csv(filepath_or_buffer, nrows=None):
    """Load the OpenFoodFacts CSV file.
    
    Parameters
    ----------
    filepath_or_buffer : str, path object or file-like object
        Valid string path to the CSV file.
    nrows : int, optional
        Number of rows that should be read ; if None, read all.
    
    Returns
    -------
    df : pd.DataFrame
        DataFrame containing the OpenFoodFacts tabular data.
    """
    # read the first 1000 lines
    df = pd.read_csv(filepath_or_buffer, sep="\t", nrows=10000, low_memory=False)
    # set explicit data types for numeric and date columns, handle the rest as text
    dtype = {}
    for col_name in df.columns.values:
        if (col_name.endswith('_n') or
            col_name in ('nutriscore_score', 'nova_group',
                         'ecoscore_score_fr', 'nutrition-score-fr_100g')):
            # columns with nullable integers : number of X (_n), some scores
            col_type = 'Int64'  # to save memory: 'Int8'
        elif col_name.endswith('_100g'):
            # nutritional values are decimal numerals
            col_type = 'float64'  # to save memory: 'float32' (close to 10-5)
        elif col_name.endswith('_t') or col_name.endswith('_datetime'):
            # Unix times and ISO 8601 datetimes will be cast explicitly later
            col_type = 'object'
        else:
            # other columns are handled as text
            col_type = 'string'
        dtype[col_name] = col_type
    # process columns with sets : _tags, states, states_en etc.
    set_columns = ('states', 'states_en')
    for col_name in set_columns:
        if col_name == 'states':
            str_sep = ', '
        else:
            str_sep = ','
        split_col = df[col_name].str.split(str_sep, expand=False)
        uniq_vals = set(itertools.chain.from_iterable(split_col))
        for uniq_val in uniq_vals:
            df[col_name + '__' + uniq_val] = split_col.apply(lambda x: uniq_val in x)
        df.drop(columns=[col_name], inplace=True)
    # re-read the file with explicit data types
    df = pd.read_csv(filepath_or_buffer, sep='\t', nrows=nrows, dtype=dtype)
    # convert columns
    for col_name in df.columns.values:
        if col_name.endswith('_t'):
            # Unix timestamps
            # NB : adding ".dt.tz_localize('UTC')" results in the same value as in the _datetime field 
            df[col_name] = pd.to_datetime(df[col_name], unit='s')
        elif col_name.endswith('_datetime'):
            # ISO 8601 dates
            df[col_name] = pd.to_datetime(df[col_name])
        elif col_name.endswith('_tags'):
            # comma-separated list of tags
            df[col_name] = df[col_name].str.split(',', expand=False)
        else:
            continue
    #
    return df

In [85]:
df = load_off_csv(OFF_FILE, nrows=10000)
# _tags + states + states_en
# df["states_tags"].values.tolist()
# import itertools
# len(set(itertools.chain.from_iterable(df["states_en"].str.split(','))))

63      True
165     True
337     True
339     True
354     True
        ... 
9349    True
9356    True
9368    True
9419    True
9570    True
Name: states__en:characteristics-completed, Length: 295, dtype: bool
1       True
2       True
3       True
4       True
5       True
        ... 
9917    True
9939    True
9966    True
9991    True
9992    True
Name: states__en:photos-uploaded, Length: 4583, dtype: bool
5       True
32      True
40      True
50      True
56      True
        ... 
9994    True
9996    True
9997    True
9998    True
9999    True
Name: states__en:ingredients-completed, Length: 6437, dtype: bool
37      True
49      True
63      True
75      True
113     True
        ... 
5012    True
5387    True
5479    True
6173    True
7416    True
Name: states__en:packaging-code-completed, Length: 81, dtype: bool
25      True
63      True
79      True
165     True
172     True
        ... 
9791    True
9797    True
9842    True
9845    True
9995    True
Name: states__en:packag

energy-kj_100g
<StringArray>
[     <NA>,   '685.8',     '936',    '1513',    '2179',       '5',     '450',
     '455',  '1952.4',    '1329',    '2318',    '1104',    '2257',    '1284',
     '177',    '1640',     '180',       '1',     '144',    '2278',     '320',
    '1900',    '1965',    '1533',    '1768',    '1670',    '1590',    '1644',
     '212',     '310',     '232',     '268',     '260',     '131',    '1896',
     '190',    '1449',       '0',      '93',    '1665',    '1809',    '1981',
    '1076',     '100',    '1253',     '137',     '196', '1245.59', '1294.66',
    '1903',    '1807',    '2030',     '525',    '1630',     '564',    '1983',
    '1986',     '587',     '188',     '615',    '8400',    '1631',    '1459',
    '1883',    '1554',    '3404',     '192',     '354',     '843',    '1618',
     '688',     '586',     '331',    '1560',    '1420',     '622',      '17',
      '89',     '811',    '1070',    '1495',     '134',      '59']
Length: 83, dtype: string


TypeError: float() argument must be a string or a number, not 'NAType'

In [90]:
dfbis["vitamin-b1_100g"].unique()

<StringArray>
[<NA>, '0']
Length: 2, dtype: string

In [65]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915377 entries, 0 to 1915376
Columns: 186 entries, code to carnitine_100g
dtypes: datetime64[ns, UTC](2), datetime64[ns](2), float64(111), object(14), string(57)
memory usage: 10.9 GB


In [48]:
df2.dtypes

code                               string
url                                string
creator                            string
created_t             datetime64[ns, UTC]
created_datetime      datetime64[ns, UTC]
                             ...         
choline_100g                      float64
phylloquinone_100g                float64
beta-glucan_100g                  float64
inositol_100g                     float64
carnitine_100g                    float64
Length: 186, dtype: object